## Initialize Feedback Function(s)

In [1]:
# Create snowpark session.

from snowflake.snowpark import Session

snowflake_connection_parameters = {
    "account": "SNOWHOUSE",
    "user": "dhuang",
    "authenticator": "externalbrowser",
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

/opt/anaconda3/envs/trulens/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
from trulens.providers.cortex import Cortex

provider = Cortex(
    model_engine="claude-4-sonnet",
    snowpark_session=snowpark_session,
    reasoning_effort="high",
)
# provider = OpenAI(model_engine="o3")

# Create feedback functions without selectors (better for direct string evaluation)
# These can be called directly on strings, which is what you want for your DataFrame processing

feedback_functions = {
    # "Logical Consistency": provider.logical_consistency_with_cot_reasons,
    # "Execution Efficiency": provider.execution_efficiency_with_cot_reasons,
    # "Plan Adherence": provider.plan_adherence_with_cot_reasons,
    # "Plan Quality": provider.plan_quality_with_cot_reasons,
    # "TRAIL": provider.trail_with_cot_reasons,
    # "Tool Usage": provider.tool_usage_with_cot_reasons,
    "Tool Selection": provider.tool_selection_with_cot_reasons,
    "Tool Calling": provider.tool_calling_with_cot_reasons,
}

for name in feedback_functions:
    print(f"- {name}")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/anaconda3/envs/trulens/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


- Tool Selection
- Tool Calling


## PROCESS ALL GAIA FILES

In [3]:
# import json
# import os

# import pandas as pd
# from trail_traversal import build_span_tree
# from trail_traversal import reset_state

# gaia_dir = "GAIA"
# all_files = []
# for filename in os.listdir(gaia_dir):
#     if filename.endswith(".json"):
#         reset_state()
#         print(filename)
#         filepath = os.path.join(gaia_dir, filename)
#         with open(filepath, "r") as f:
#             trace_data = json.load(f)
#         output_file = os.path.join(gaia_dir, filename.replace(".json", ".txt"))
#         with open(output_file, "w") as f:
#             f.write(f"Trace ID: {trace_data['trace_id']}\n\n")
#         root_spans, all_spans = build_span_tree(trace_data)
#         for root_span in root_spans:
#             root_span.display(output_file=output_file)
#         all_files.append(output_file)

# print(len(all_files))

## RUN EVALS

In [4]:
GAIA_trace_explanation = """
Agent Architecture and Trace Structure: The agent architecture consists of a primary manager Agent (also referred to as CodeAgent) that delegates tasks to a search_agent (also referred to as ToolCallingAgent).

Overall Flow:
Every trace consists of several spans (with span_id numbers and parent span_id numbers). Each trace begins with the manager (CodeAgent). The process follows a clear, hierarchical structure where the manager outlines a high-level plan and the search_agent executes the detailed, tool-based steps for each part of that plan.

1. Manager Agent Initiation:
The trace starts with the manager. In its initial child spans, you will observe the following sequence:
- A preparatory survey is created based on the user's query.
- A high-level plan is formulated from this survey.

The Manager agent begins executing Step 1 of its plan.

2. Manager Agent Step 1:
Within the child span for Step 1, the Manager agent decides how to proceed given the initial fact survey and plan. The Manager agent will produce a thought, which may call the search_agent to perform the necessary actions or research.

3. search_agent (ToolCallingAgent) Execution Loop:
Once called, the search_agent begins its own execution loop. In its child spans, you will observe the following sequence:
- A preparatory survey to the specific sub-task it received from the Manager agent.
- A plan tailored to the specific sub-task it received from the Manager agent.

The search_agent executes an initial set of up to four steps. Each step involves an LLM call to generate a tool-call, followed by the tool's execution.
After these initial steps, search_agent synthesizes the information gathered into an updated fact list and refines its plan.
The search_agent may then continue to execute more tool-steps based on this updated plan.

This loop continues until the search_agent has gathered enough information to comprehensively answer the manager's sub-task, at which point it calls final_answer.

4. Returning Control to the Manager agent
The final_answer from the search_agent is returned to the Manager agent, concluding the Manager agent's Step 1. The Manager agent then proceeds to Step 2 of its high-level plan, using the result from the previous step as context. 
This entire cycle repeats for all subsequent steps in the Manager Agent's plan.

Whenever you want to point out anything in the trace, cite the span_id number of the span that you are referring to.
"""

In [5]:
# logical_consistency_prompt = """
# Track each agent's system instructions and conversation history, ensuring all subsequent outputs from that agent adhere to its established guidelines and prior dialogue, even when agents speak interchangeably. 
# For the manager agent and each unique search_agent that may exist in the trace, evaluate the logical consistency for the agent's actions and responses. For each agent, ensure that each response is consistent with the system instructions and prior dialogue.
# You must structure your entire response: 
# **Manager Agent** 
# **System Instructions**
# [Paste all system instructions associated with the manager agent.]
# **Logical Consistency issues**
# [All Logical Consistency issues associated with the manager agent] 

# **search_agent 0** (if exists)
# **System Instructions**
# [Paste all system instructions associated with the search_agent.]
# **Logical Consistency issues**
# [List all Logical Consistency issues associated with this search_agent] 
# ... 
# **search_agent n** (if exists)
# **System Instructions**
# [Paste all system instructions associated with the search_agent.]
# **Logical Consistency issues**
# [List all Logical Consistency issues associated with this search_agent]

# Here are some examples of logical consistency issues:
#         {
#             "evidence": "The plan output content ends with the last step of the plan instead of the `<end_plan>` tag.",
#             "description": "The plan generation step did not conclude with the required '<end_plan>' tag as specified in the instructions for plan generation.",
#         },
#         {
#             "evidence": "Thought: I recall that Girls Who Code published a statistic showing that in the 1980s, women accounted for 37% of computer scientists (or computer science graduates) and that over a span of three decades that figure dropped by 13 percentage points (from 37% down to 24%)",
#             "description": "The system recalled a statistic about Girls Who Code and the percentage of women in computer science, but this information was not verified using the search tool as planned. The system states \"Thought: I recall that\nGirls Who Code published a statistic showing that in the 1980s, women accounted for 37% of computer scientists (or computer science graduates) and that over a span of three decades that figure dropped by 13 percentage points\n(from 37% down to 24%). In other words, it took 30 years for that change to occur. Based on that well-circulated statistic that Girls Who Code highlighted, I will output 30 years as the final answer."
#         }
# Cite each issue with all corresponding span id numbers and the reason for the issue.
# """

# execution_efficiency_prompt = """
# Track each agent's system instructions and conversation history, ensuring all subsequent outputs from that agent adhere to its established guidelines and prior dialogue, even when agents speak interchangeably. 
# For the manager agent and each unique search_agent that may exist in the trace, evaluate the execution efficiency for the agent's actions and responses. 
# You must structure your entire response: 
# **Manager Agent** 
# [List each execution efficiency issue associated with the manager agent with an explanation and citation(s)] 

# **search_agent 0** (if exists)
# [List each execution efficiency issue associated with this search_agent with an explanation and citation(s)] 
# ...
# **search_agent n** (if exists)
# [List each execution efficiency issue associated with this search_agent with an explanation and citation(s)] 

# Here are some examples of execution efficiency issues:
#         {
#             "evidence": "{'input.value': '{\"args\": [], \"sanitize_inputs_outputs\": true, \"kwargs\": {\"\": \"\"}}', 'openinference.span.kind': 'TOOL', 'pat.app': 'GAIA-Samples', 'pat.project.id': 'a69d64fc-5115-468e-95ed-0950bd37f06a', 'pat.project.name': 'gaia-annotation-samples', 'tool.description': 'Scroll the viewport DOWN one page-length in the current webpage and return the new viewport content.', 'tool.name': 'page_down', 'tool.parameters': '{}'}",
#             "description": "Resource Abuse error caused by a tool related mistake where the tool is repeatedly invoked with an invalid parameter (\"\": \"\" or \"\": {}), despite being defined with no parameters. This repeated misuse signals abnormal or excessive use of the tool with incorrect input, triggering a Resource Abuse error.",
#         }
# Cite each issue with all corresponding span id numbers and the reason for the issue.
# """

# plan_quality_prompt = """
# Look for the keyword '[PLAN]' to identify plans for the manager agent and each unique search_agent that may exist in the trace. 
# Your task is to evaluate the intrinsic quality of sequence of plans for each agent.
# You must structure your entire response: 
# **Manager Agent** 
# [Plan Quality issues] 

# **search_agent 0** (if exists)
# [Plan Quality issues] 

# ... 
# **search_agent n** (if exists)
# [Plan Quality issues]

# Here are some examples of plan quality issues:
#     {
#             "evidence": "1. Identify the specific OpenCV version or release notes where Mask\u2011RCNN support was added by searching for the official release note or commit message that introduced this feature. \n2. Retrieve the commit history or changelog details for that version to determine the list of contributors responsible for adding Mask\u2011RCNN support. \n3. Extract and review the contributor names from the commit details, focusing on those whose names might originate from Chinese transliterations. \n4. Research a reliable list of former Chinese heads of government with their names transliterated into the Latin alphabet. \n5. Compare and cross-match the contributor names with the list of former Chinese heads of government to identify the one whose Latin name exactly matches. \n6. Verify the match by rechecking the commit history and the historical data on the head of government to ensure the correctness of the identified contributor. \n7. Conclude with the final contributor\u2019s name as the correct answer.",
#             "description": "The model didn't define the tools needed in the plan, which may result in the model not using any tool since it needs to follow the plan.",
#         },
#         {
#             "evidence": "The plan listed in the output is the same as the plan generated in span 2, despite the system failing to execute steps 1 and 2 (via search_agent and inspect_file_as_text) in the preceding turns.",
#             "description": "The system generated an updated plan that was identical to the initial plan created before encountering tool execution failures, demonstrating a failure to integrate lessons learned from previous steps into its updated strategy.",
#         },

# Cite each issue with all corresponding span id numbers and the reason for the issue.
# """

# plan_adherence_prompt = """
# Look for the keyword '[PLAN]' to identify plans for the manager agent and each unique search_agent that may exist in the trace. 
# Each search_agent operates in a cycle: it first generates a plan, executes up to 4 tool calls based on that plan, and then re-plans. Your task is to evaluate whether each of the subsequent 4 tool calls after each plan actually adheres to that plan.
# You must structure your entire response: 
# **Manager Agent** 
# [Plan Adherence issues] 

# **search_agent 0** (if exists)
# [Plan Adherence issues] 
# ... 
# **search_agent n** (if exists)
# [Plan Adherence issues]

# Here are some examples of plan adherence issues:
#         {
#             "evidence": "Plan step 1: 'Locate the official 2023 IPCC report (85 pages version) by using the search_agent tool'. Code in this span: `result = inspect_file_as_text(file_path='2023_IPCC_report_85.pdf', ...)`",
#             "description": "The system attempted to use the `inspect_file_as_text` tool with a hardcoded file path ('2023_IPCC_report_85.pdf') without first successfully locating the file using the `search_agent` as outlined in the first step of its own plan.",
#         }
#         {
#             "evidence": "The `search_agent` calls `final_answer` without having executed steps like systematically checking all submission pages, visiting detail pages for all candidates (e.g. Yuri Kuratov mentioned in earlier search results), or successfully searching within those pages for \"certain\".",
#             "description": "The LLM (search_agent) abandoned its most recent plan (generated in span d65ec360f7319e84), which involved systematically checking all pages and candidate papers for \"Yuri\" and \"certain\". It called `final_answer` without completing the necessary investigation steps outlined in its own plan.",
#         }
        
# Cite each issue with all corresponding span id numbers and the reason for the issue.
# """

## Tool Usage eval


In [6]:
# tool_usage_prompt = """
# Track each agent's tools, system instructions, and conversation history, ensuring all subsequent tool calls and interpretations adhere to the tools’ definitions and prior dialogue, even when agents speak interchangeably.
# For the manager agent and each unique search_agent that may exist in the trace, evaluate the QUALITY OF TOOL USAGE (reasoning around tools). Do NOT score workflow efficiency (covered by Execution Efficiency) or plan deviations (covered by Plan Adherence).

# You must structure your entire response:

# **Manager Agent**
# **Tool Descriptions**
# [Paste verbatim every tool available to the manager agent, including: tool.name, tool.description, and tool.parameters (schema/required args). If a tool named `final_answer` exists, list it as a tool. If no tools are defined for this agent, write: "No tools found."]

# **Tool Usage Issues**
# [List each tool-usage issue for the manager agent with an explanation and citation(s). Focus on: Selection, Input Quality, Output Integration, Context Handling, Error Robustness, Transparency.]

# **search_agent 0** (if exists)
# **Tool Descriptions**
# [Paste verbatim every tool available to this search_agent, including: tool.name, tool.description, and tool.parameters. If `final_answer` exists as a tool, list it. If no tools are defined, write: "No tools found."]

# **Tool Usage Issues**
# [List each tool-usage issue for this search_agent with an explanation and citation(s).]

# ...
# **search_agent n** (if exists)
# **Tool Descriptions**
# [Paste verbatim every tool available to this search_agent, including: tool.name, tool.description, and tool.parameters. If `final_answer` exists as a tool, list it. If no tools are defined, write: "No tools found."]

# **Tool Usage Issues**
# [List each tool-usage issue for this search_agent with an explanation and citation(s).]

# Scoring Scope (what to judge here):
# - Selection: Was the chosen tool appropriate/necessary vs. a better available option? Any misuse or irrelevant calls?
# - Input Quality: Were arguments/queries precise, complete, and well-scoped? Were failed inputs reformulated thoughtfully?
# - Output Integration: Were tool outputs interpreted faithfully and used correctly in subsequent reasoning/actions? Any fabrications or ignoring of results?
# - Context Handling: Did the agent use prior instructions, tool outputs, and tool specs to avoid repeating mistakes or contradictions?
# - Error Robustness: Were empty/irrelevant results and API/service errors explicitly recognized and handled with adaptive strategies?
# - Transparency: Did the agent briefly explain why a tool was used and how the output influenced next steps?

# EXCLUDE from this judge (handled elsewhere):
# - Workflow redundancy, excessive retries, poor sequencing → Execution Efficiency.
# - Deviations from written plan steps → Plan Adherence.

# Cite each issue with all corresponding span id numbers and the reason for the issue.

# Here are some examples of TOOL USAGE issues:
#     {
#         "evidence": "Agent uses `python_interpreter` to 'search the web' despite `search_agent` being available and defined for web queries.",
#         "description": "Selection: Wrong tool chosen for the subtask; better-suited tool (`search_agent`) was available and documented.",
#         "spans": ["0242ca2533f.."]
#     },
#     {
#         "evidence": "Tool output shows 'No results found', but the agent proceeds to assert a specific value sourced 'from the tool'.",
#         "description": "Output Integration: Misinterprets/contradicts tool output; fabricates a value not present in results.",
#         "spans": ["0035f455b.."]
#     },
#     {
#         "evidence": "After receiving a 403 Authentication error, the agent retries the same call with identical credentials and no acknowledgement.",
#         "description": "Error Robustness / Context Handling: Fails to recognize and handle auth errors; no adaptation (e.g., re-auth, alternative data source, or deferring).",
#         "spans": ["7ee8e8df6.."]
#     },
#     {
#         "evidence": "Plan step: 'Use `search_agent` then `inspect_file_as_text`'. The agent calls `inspect_file_as_text` with a guessed path without first locating the file; later claims 'results confirm'.",
#         "description": "Context Handling / Output Integration: Ignores prerequisite context for the tool and overclaims about results not returned by the tool.",
#         "spans": ["plan-span-d65ec360..", "exec-span-abc123.."]
#     },
#     {
#         "evidence": "The agent says: 'Using the dataset tool to verify,' but no call to the dataset tool appears in the trace.",
#         "description": "Transparency: Claims to use a tool without an actual invocation; creates an audit gap between stated and observed tool use.",
#         "spans": ["0140b3f65.."]
#     }

# Additional directives:
# - In **Tool Descriptions**, extract only from the trace; do not invent missing schema. Preserve parameter names and required/optional status if present.
# - If a tool alias or wrapper is used, note both the alias and the underlying tool if the trace reveals it.
# - If multiple versions of a tool definition appear, list all and indicate which one applies at each span (if discernible).
# - When in doubt whether `final_answer` is a tool or a control, check the trace metadata. If it is a tool invocation in the trace, treat it as a tool.
# """

## Tool evals judge custom instructions

In [7]:

tool_selection_prompt = """
Track each agent's system instructions, available tools, and conversation history. Your task is to evaluate whether the agent SELECTED the most appropriate tools for its stated tasks/subtasks, given the tool descriptions and parameters.
Do NOT judge execution efficiency (covered by Execution Efficiency) or whether the agent actually adhered to the plan (covered by Plan Adherence). Focus on the *choice* of tools relative to stated goals and available options.

You must structure your entire response:

**Manager Agent**
**Tool Descriptions**
[Paste verbatim every tool available to the manager agent, including: tool.name, tool.description, tool.parameters/schema and required args. If a tool named `final_answer` exists as an invocable tool, list it. If no tools are defined, write: "No tools found."]

**Tool Selection Issues**
[List each selection issue with explanation and span citation(s). If the agent chose to do something internally where a tool was clearly superior or required by instructions, flag it. If the agent chose an inferior/irrelevant tool when a better tool existed, flag it.]

**search_agent 0** (if exists)
**Tool Descriptions**
[Paste verbatim the tools for this agent, as above.]

**Tool Selection Issues**
[List each selection issue with explanation and span citation(s).]

...
**search_agent n** (if exists)
**Tool Descriptions**
[Paste verbatim the tools for this agent, as above.]

**Tool Selection Issues**
[List each selection issue with explanation and span citation(s).]

Scoring Scope (what to judge here):
- Match-to-goal: For each task/subtask the agent undertakes, did it pick the best-suited tool from those available?
- Comparative suitability: If multiple tools could work, did it choose the one with clearer preconditions/postconditions, more direct support, or stricter guarantees?
- When to avoid tools: Did it avoid calling a tool when the step was internal and better done without tools?
- Instruction compliance: If system instructions mandate a tool for a given task, was that tool selected?
- Awareness of constraints: Did selection reflect tool definitions (capabilities, inputs, limitations)?

EXCLUDE from this judge:
- Whether arguments were correct or outputs were interpreted faithfully → Tool Calling.
- Resource waste, retries, sequencing inefficiency → Execution Efficiency.
- Whether steps in the plan were followed → Plan Adherence.

Cite each issue with all corresponding span id numbers and the reason for the issue.

Examples of Tool Selection issues:
    {
        "evidence": "The agent used `python_interpreter` to perform web search despite `search_agent` being defined for browsing.",
        "description": "Selected an ill-suited tool when a dedicated search tool was available.",
        "spans": ["0242ca2533f.."]
    },
    {
        "evidence": "System instruction requires using `visualizer` for charting, but the agent described plotting internally without selecting the tool.",
        "description": "Failed to select a mandated tool per instructions.",
        "spans": ["1427b326.."]
    },
    {
        "evidence": "Task: 'inspect the PDF text'. Tools available: `inspect_file_as_text` (PDF text extraction), `final_answer`. Agent selected `final_answer` directly.",
        "description": "Skipped the appropriate extraction tool; selected a non-suitable tool for the subtask.",
        "spans": ["08be1639.."]
    }
"""

tool_calling_prompt = """
Track each agent's system instructions, available tools, and conversation history. Your task is to evaluate the QUALITY OF TOOL CALLS made by the agent that are within the agent’s control:
- Were inputs (arguments/queries) syntactically valid and semantically appropriate given the tool’s description, parameters, preconditions, and expected postconditions?
- Did the agent correctly interpret and integrate the tool outputs?

Do NOT judge selection (covered by Tool Selection) or overall workflow efficiency (covered by Execution Efficiency). Focus on *how* the tool was called and how its outputs were handled.

You must structure your entire response:

**Manager Agent**
**Tool Descriptions**
[Paste verbatim every tool available to the manager agent, including: tool.name, tool.description, tool.parameters/schema and required args. If `final_answer` is an invocable tool, list it. If no tools are defined, write: "No tools found."]

**Tool Calling Issues**
[List each tool-calling issue for the manager agent with explanation and span citation(s). Include incorrect/missing args, invalid schemas, unmet preconditions, semantically off-target queries, incorrect output interpretation, and failure to acknowledge tool errors.]

**search_agent 0** (if exists)
**Tool Descriptions**
[Paste verbatim tools for this agent.]

**Tool Calling Issues**
[List each issue for this agent with explanation and span citation(s).]

...
**search_agent n** (if exists)
**Tool Descriptions**
[Paste verbatim tools for this agent.]

**Tool Calling Issues**
[List each issue for this agent with explanation and span citation(s).]

Scope boundaries:
- In-scope: Syntactic validity, argument completeness, semantic appropriateness of queries, honoring required params, satisfying preconditions, correct parsing/grounded use of outputs, explicit handling of tool-returned errors (recognition + appropriate adaptation).
- Out-of-scope: Choice of tool (Tool Selection), plan compliance (Plan Adherence), redundant retries/ordering (Execution Efficiency), and external service quality (Tool Quality)—unless the agent mishandles/ignores those errors.

Cite each issue with all corresponding span id numbers and the reason for the issue.

Examples of Tool Calling issues:
    {
        "evidence": "tool.name: 'page_down' with parameters {}. Calls show args: {'': ''} repeatedly.",
        "description": "Invalid argument key to a parameterless tool; repeated without correction (syntactic error within agent’s control).",
        "spans": ["041b7f9c..", "041b7f9c..-retry2"]
    },
    {
        "evidence": "search tool returned 'No results', yet agent asserts a specific fact 'from the tool'.",
        "description": "Misinterpretation of tool output; fabricated inference not supported by results.",
        "spans": ["0035f455b.."]
    },
    {
        "evidence": "Agent queries `cortex_search` with 'salary' while task requires '2024 US base pay bands for L5'; no reformulation after irrelevant results.",
        "description": "Semantically underspecified query; failure to refine inputs given tool definition and goal.",
        "spans": ["0242ca2533f.."]
    }
"""

tool_quality_prompt = """
Independently evaluate the QUALITY and RELIABILITY of the TOOLS themselves as observed in the trace, conditioned on known parameters, preconditions, and postconditions. This judge attributes *external/system/tool* errors to the tool quality dimension rather than to agent behavior.

Your job is NOT to judge the agent’s choices or call syntax, but to assess how each tool performed: availability, stability, response quality, latency/reliability, and domain-specific metrics (e.g., search relevance for a search tool).

You must structure your entire response:

**Manager Agent**
**Tool Descriptions**
[Paste verbatim every tool available to the manager agent, including: tool.name, tool.description, tool.parameters/schema and required args. If `final_answer` is an invocable tool, list it. If no tools are defined, write: "No tools found."]

**Observed Calls and Outcomes**
[Summarize observed calls to each tool (name + spans), notable responses (e.g., 5xx/4xx, timeouts, empty/irrelevant payloads), and any meta signals (latency if present).]

**Tool Quality Findings**
[List quality issues attributable to the tool or external system with explanation and span citation(s). Include: Service Errors (5xx), Rate Limiting (429), Auth (401/403), Resource Not Found (404), Timeout, flaky/determinism issues, and domain-specific poor quality (e.g., low search relevance).]

**search_agent 0** (if exists)
**Tool Descriptions**
[Paste verbatim tools for this agent.]

**Observed Calls and Outcomes**
[As above.]

**Tool Quality Findings**
[As above.]

...
**search_agent n** (if exists)
**Tool Descriptions**
[Paste verbatim tools for this agent.]

**Observed Calls and Outcomes**
[As above.]

**Tool Quality Findings**
[As above.]

Scope boundaries:
- In-scope: External/system/tool-side issues and intrinsic output quality (e.g., search relevance, result completeness), given the inputs supplied.
- Out-of-scope: Whether the agent selected the right tool (Tool Selection), whether arguments were well-formed (Tool Calling), or workflow inefficiency (Execution Efficiency). If an apparent failure is due to clearly invalid inputs, note it briefly but do not penalize Tool Quality—penalize in Tool Calling.

Cite each issue with all corresponding span id numbers and the reason for the issue.

Examples of Tool Quality issues:
    {
        "evidence": "HTTP 500 Service Error returned by `web_fetch` despite valid args; multiple spans show intermittent 5xx.",
        "description": "Service instability reduces tool reliability (external error).",
        "spans": ["7ee8e8df6..", "7bf0addd.."]
    },
    {
        "evidence": "Search tool returns top-5 results irrelevant to query intent across consecutive calls despite distinct, specific queries.",
        "description": "Low search relevance indicates tool output quality issue (domain-specific quality).",
        "spans": ["14be0e98b.."]
    },
    {
        "evidence": "Repeated 429 rate limiting despite modest call frequency; backoff hints missing from tool response.",
        "description": "Rate limiting sensitivity lowers effective availability.",
        "spans": ["0035f455b..-rate"]
    }
"""


## TEST SINGULAR TRACE

In [ ]:
GAIA_FILE_PREFIX = "/Users/dhuang/Documents"

test_file = f"{GAIA_FILE_PREFIX}/GAIA/041b7f9c8c76c2ca1a8e67c6769267c3.txt"

with open(test_file, "r") as f:
    test_data = f.read()

for i, (feedback_name, feedback_func) in enumerate(feedback_functions.items()):
    if feedback_name == "Tool Selection":
        result = feedback_func(
            test_data,
            custom_instructions=GAIA_trace_explanation + tool_selection_prompt,
        )
    if feedback_name == "Tool Calling":
        result = feedback_func(
            test_data,
            custom_instructions=GAIA_trace_explanation + tool_calling_prompt,
        )
    if isinstance(result, tuple) and len(result) == 2:
        score, metadata = result
        reason = metadata.get("reason", "")
        print(f"{feedback_name}: {score} {reason}")

Tool Selection: 0.0 Criteria: Tool Selection evaluates whether the agent selected the most appropriate tools for its stated tasks/subtasks, given the tool descriptions and parameters available.
Supporting Evidence: The manager agent had a clear plan to use search_agent to find the number of Nature articles published in 2020, but failed to execute this critical step. In span fb10fb02e8732571, the agent prepared a task string for search_agent but only printed it using python_interpreter. In span 1832b9469b9b862d, the agent again used python_interpreter instead of the designated search_agent tool. Most critically, in span 227551d8e97e8f38, the agent proceeded with calculations using an unverified assumption (484 articles) without ever calling search_agent, directly contradicting the plan's first step. The agent had access to search_agent specifically designed for web research but chose to use python_interpreter for tasks it wasn't suited for, and then made up data rather than using the ap

## RUN ON ALL TRACES

In [9]:
import os

from sklearn.model_selection import train_test_split

all_files = []
gaia_dir = "/Users/dhuang/Documents/GAIA"
for filename in os.listdir(gaia_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(gaia_dir, filename)
        all_files.append(filepath)

all_files.sort()

train_files, test_files = train_test_split(
    all_files, test_size=0.5, random_state=42
)
train_files.sort()
test_files.sort()
print(f"train_files: {train_files}")
print(f"test_files: {test_files}")

train_files: ['/Users/dhuang/Documents/GAIA/0140b3f657eddf76ca82f72c49ac8e58.txt', '/Users/dhuang/Documents/GAIA/01c5727165fc43899b3b594b9bef5f19.txt', '/Users/dhuang/Documents/GAIA/0242ca2533fac5b8b604a9060b3e15d6.txt', '/Users/dhuang/Documents/GAIA/0adc4f3b99d9564d32811e913cc9d248.txt', '/Users/dhuang/Documents/GAIA/1427b326e21963a1228647ad8dff2bf4.txt', '/Users/dhuang/Documents/GAIA/2209204b631e26f4cee4dc3dd995608c.txt', '/Users/dhuang/Documents/GAIA/25c8275651013fe8398ef0f735eb0912.txt', '/Users/dhuang/Documents/GAIA/2c77a8feec544cc61a00a387ad792a13.txt', '/Users/dhuang/Documents/GAIA/3215fc75e81bdb73706a4fb37b66427f.txt', '/Users/dhuang/Documents/GAIA/331aece579d942bb4c345a86c86efb83.txt', '/Users/dhuang/Documents/GAIA/3637c5845140adbf29c565923c20e94d.txt', '/Users/dhuang/Documents/GAIA/396b6aa1ab86eb2e20d27582eb5eebd9.txt', '/Users/dhuang/Documents/GAIA/41bbc898aa7de0f31d2382ff57700a76.txt', '/Users/dhuang/Documents/GAIA/4ae16319f0de44a7d1e84595b41ae08d.txt', '/Users/dhuang/Docum

In [15]:
split = "test"
if split == "train":
    all_files = train_files
else:
    all_files = test_files

csv_path = f"tool_judges_0910_{split}.csv"

In [16]:
csv_path

'tool_judges_0910_test.csv'

In [17]:
import pandas as pd

all_results = []

print(f"Status: {len(all_results)} completed, {len(all_files)} remaining")
print(f" Next files to process: {all_files[:3]}")

# Process remaining files one by one
for i, file in enumerate(all_files):
    file_name = file.split(".")[0]
    print(f"\n{'=' * 60}")
    print(f"Processing {i + 1}/{len(all_files)}: {file_name}")
    print(f"{'=' * 60}")

    try:
        # Read file
        with open(file, "r") as f:
            gaia_file = f.read()

        results = {"filename": file_name}

        # Process each feedback function
        for j, (feedback_name, feedback_func) in enumerate(
            feedback_functions.items()
        ):
            print(
                f"[{j + 1}/{len(feedback_functions)}] Evaluating: {feedback_name}"
            )

            try:
                if feedback_name == "Tool Selection":
                    result = feedback_func(
                        gaia_file,
                        custom_instructions=GAIA_trace_explanation
                        + tool_selection_prompt,
                    )
                if feedback_name == "Tool Calling":
                    result = feedback_func(
                        gaia_file,
                        custom_instructions=GAIA_trace_explanation
                        + tool_calling_prompt,
                    )
                if isinstance(result, tuple) and len(result) == 2:
                    score, metadata = result
                    results[f"{feedback_name}_score"] = score
                    reason = metadata.get("reason", "")
                    results[f"{feedback_name}_reasons"] = reason
                    print(f"Score: {score}")
                else:
                    print("Unexpected result format")

            except Exception as e:
                print(f"Error: {str(e)[:100]}...")
                results[f"{feedback_name}_score"] = None
                results[f"{feedback_name}_reasons"] = f"Error: {str(e)[:200]}"

        # Add to results and save immediately
        all_results.append(results)

        results_df = pd.DataFrame([results])
        results_df.to_csv(
            csv_path, mode="a", header=not os.path.exists(csv_path), index=False
        )
        print(
            f"Completed {file_name} | Total: {len(all_results)}/{len(all_files)}"
        )

    except Exception as e:
        print(f"FAILED {file_name}: {e}")
        continue

# Final save
print(f"\nFinished processing {len(all_results)} files")
print(all_results)
# final_df = pd.DataFrame(all_results)
# final_df.to_csv("trail_benchmark_aug18.csv", index=False)
# print("Final results saved to: trail_benchmark_aug18.csv")

Status: 0 completed, 59 remaining
 Next files to process: ['/Users/dhuang/Documents/GAIA/0035f455b3ff2295167a844f04d85d34.txt', '/Users/dhuang/Documents/GAIA/041b7f9c8c76c2ca1a8e67c6769267c3.txt', '/Users/dhuang/Documents/GAIA/08be1639c58e086cf0bb8c269039973d.txt']

Processing 1/59: /Users/dhuang/Documents/GAIA/0035f455b3ff2295167a844f04d85d34
[1/2] Evaluating: Tool Selection
Score: 1.0
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/0035f455b3ff2295167a844f04d85d34 | Total: 1/59

Processing 2/59: /Users/dhuang/Documents/GAIA/041b7f9c8c76c2ca1a8e67c6769267c3
[1/2] Evaluating: Tool Selection
Score: 0.0
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/041b7f9c8c76c2ca1a8e67c6769267c3 | Total: 2/59

Processing 3/59: /Users/dhuang/Documents/GAIA/08be1639c58e086cf0bb8c269039973d
[1/2] Evaluating: Tool Selection
Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/08be1639c5

Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Multiple valid rating values found in the string: **Initial Plan Identification**
1. Use the inspect_file_as_text tool to extract the transcription from the provided MP3 audio file.
2. Parse the transcription to identify the specific original line that needs to be transformed into an anagram.
3. Analyze the professor's instructions to understand that the final output must be the anagram (with no punctuation) generated from the original line.
4. Generate the anagram by rearranging the letters of the original line ensuring that every letter is used exactly once.
5. Verify that the generated anagram meets all conditions: it uses exactly the same letters as the original, contains no punctuation, and follows the professor's requirements.
6. Submit the final answer (the anagram text only) using the final_answer tool.

**Replan Identification**
No explicit replan w

Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/512475a321c616e45337da3575f6a185 | Total: 21/59

Processing 22/59: /Users/dhuang/Documents/GAIA/53dba4241b22d5039c9c119871c7c8b4
[1/2] Evaluating: Tool Selection
Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/53dba4241b22d5039c9c119871c7c8b4 | Total: 22/59

Processing 23/59: /Users/dhuang/Documents/GAIA/59365b27641e501d105b0e8f5e7c5af7
[1/2] Evaluating: Tool Selection
Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/59365b27641e501d105b0e8f5e7c5af7 | Total: 23/59

Processing 24/59: /Users/dhuang/Documents/GAIA/5bbd1534b199c57861f55b58be9949a0
[1/2] Evaluating: Tool Selection
Score: 1.0
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/5bbd1534b199c57861f55b58be9949a0 | Total: 24/59

Process

CortexEndpoint request failed <class 'AttributeError'>='Response' object has no attribute 'data'. Retries remaining=3.


Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/7c98d39b2699d41fa913f8fbe60c04ed | Total: 32/59

Processing 33/59: /Users/dhuang/Documents/GAIA/7ee8e8df6e8cd101d9af8a4a4f6ceedb
[1/2] Evaluating: Tool Selection
Score: 1.0
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/7ee8e8df6e8cd101d9af8a4a4f6ceedb | Total: 33/59

Processing 34/59: /Users/dhuang/Documents/GAIA/911e853f02d03e976dbf0c16f653ab57
[1/2] Evaluating: Tool Selection
Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/911e853f02d03e976dbf0c16f653ab57 | Total: 34/59

Processing 35/59: /Users/dhuang/Documents/GAIA/96be1bf61f2915e52ec59173b9bd9828
[1/2] Evaluating: Tool Selection
Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/96be1bf61f2915e52ec59173b9bd9828 | Total: 35/59

Processing 36/59: /Use

Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Rating must be in [0, 3].
Multiple valid rating values found in the string: **Initial Plan Identification**
1. Request from the search_agent the detailed records (or dataset file) of the NCATS PubChem compound database for Food Additive Status classification, ensuring to retrieve data on molecular weight, heavy atom count, hydrogen bond acceptors, and complexity values, along with enzyme transformation and gene-chemical co-occurrence information.
2. Parse the obtained dataset to filter compounds based on the specific criteria: molecular weight of 100 g/mol or less, exactly 6 heavy atoms, 1 or fewer hydrogen bond acceptors, and a complexity between 10 and

Score: 0.3333333333333333
[2/2] Evaluating: Tool Calling
Score: -0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/a7017f40866dd77c6c0c0e98bad17f69 | Total: 43/59

Processing 44/59: /Users/dhuang/Documents/GAIA/aa76e85e274bb7187a186c5ed9c90b43
[1/2] Evaluating: Tool Selection
Score: 1.0
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/aa76e85e274bb7187a186c5ed9c90b43 | Total: 44/59

Processing 45/59: /Users/dhuang/Documents/GAIA/ae345023ab5e1c09d49c2b2c6e311877
[1/2] Evaluating: Tool Selection
Score: 0.0
[2/2] Evaluating: Tool Calling
Score: 0.0
Completed /Users/dhuang/Documents/GAIA/ae345023ab5e1c09d49c2b2c6e311877 | Total: 45/59

Processing 46/59: /Users/dhuang/Documents/GAIA/b241cb7deedf9646f01fa15095ed96d2
[1/2] Evaluating: Tool Selection
Score: 0.6666666666666666
[2/2] Evaluating: Tool Calling
Score: 0.3333333333333333
Completed /Users/dhuang/Documents/GAIA/b241cb7deedf9646f01fa15095ed96d2 | Total: 46/59

Processing 47/59: /Us

In [ ]:
test_df_with_tool_usage = pd.read_csv(
    "~/Downloads/NEW_all_eval_trail_benchmark - NEW_test_eval_trail_benchmark.csv"
)
train_df_with_tool_usage = pd.read_csv(
    "~/Downloads/NEW_all_eval_trail_benchmark - NEW_TRAIN_eval_trail_benchmark.csv"
)

In [ ]:
for index, row in test_df_with_tool_usage.iterrows():
    orig_ground_truth = row["ground truth annotations"]
    tool_usage_cot = row["Tool Usage_reasons"]
    # print(orig_ground_truth)
    analyze_prompt = f"""
    For each of these ground truth errors, you see we already had existing paragraph below: \n 
    {orig_ground_truth}
    
    the above paragraph analyzes whether
    any of the existing LLM-judge (logical consistency, execution efficiency, plan adherence, plan quality) covers the ground truth error in its own explanation. Repeat each of the ground truth errors, include the name(s) of which judges discuss that error in its explanation, and explain your reasoning. Be critical in your evaluation.
    now I want you to critically analyze the new judge Tool Usage's cot, whic is shown at the end, and see if it covers any of the ground truth errors and update them accordingly. don't remove existing lines like ""impact": "LOW" (LOGICAL CONSISTENCY - CAUGHT)", but rather you should append.
    
    Tool Usage's cot: \n
    {tool_usage_cot}
    """

    analysis_resp = provider._create_chat_completion(
        messages=[{"role": "user", "content": analyze_prompt}]
    )
    # print(analysis_resp)

    # Add the analysis to a new column for this row
    test_df_with_tool_usage.at[
        index, "error coverage analysis with tool usage judge"
    ] = analysis_resp

In [ ]:
test_df_with_tool_usage.to_csv(
    "new_test_benchmark_with_tool_usage_judge_analysis.csv", index=False
)

In [ ]:
for index, row in train_df_with_tool_usage.iterrows():
    orig_ground_truth = row["ground truth annotations"]
    tool_usage_cot = row["Tool Usage_reasons"]
    # print(orig_ground_truth)
    analyze_prompt = f"""
    For each of these ground truth errors, you see we already had existing paragraph below: \n 
    {orig_ground_truth}
    
    the above paragraph analyzes whether
    any of the existing LLM-judge (logical consistency, execution efficiency, plan adherence, plan quality) covers the ground truth error in its own explanation. Repeat each of the ground truth errors, include the name(s) of which judges discuss that error in its explanation, and explain your reasoning. Be critical in your evaluation.
    now I want you to critically analyze the new judge Tool Usage's cot, whic is shown at the end, and see if it covers any of the ground truth errors and update them accordingly. don't remove existing lines like ""impact": "LOW" (LOGICAL CONSISTENCY - CAUGHT)", but rather you should append.
    
    Tool Usage's cot: \n
    {tool_usage_cot}
    """

    analysis_resp = provider._create_chat_completion(
        messages=[{"role": "user", "content": analyze_prompt}]
    )
    # print(analysis_resp)

    # Add the analysis to a new column for this row
    train_df_with_tool_usage.at[
        index, "error coverage analysis with tool usage judge"
    ] = analysis_resp

In [ ]:
train_df_with_tool_usage.to_csv(
    "new_train_benchmark_with_tool_usage_judge_analysis.csv", index=False
)

## SWE-BENCH tool usage eval

In [ ]:
# Description of agent architecture and trace structure
SWEBench_trace_explanation = """
Agent Architecture and Trace Structure: The agent architecture consists of a CodeAgent that has access to a sandboxed environment, a python interpreter, and the "gitingest" library that can turn any Git reposistory into a text digest of its codebaes.

Overall Flow:
Every trace consists of several spans (with span_id numbers and parent span_id numbers). Each trace begins with the CodeAgent which performs actions through a cycle of steps, with existing variables and knowledge being incorporated into the agent’s context. Specifically, the CodeAgent will plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.  

At each step, in the 'Thought:' sequence, the CodeAgent should first explain its reasoning towards solving the task and the tools that it wants to use. Then in the 'Code:' sequence, it should write the code in simple Python. The code sequence must end with '<end_code>' sequence. During each intermediate step, the CodeAgent can use 'print()' to save whatever important information it will then need. These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
Each tool call and tool response will also be shown in each step. 

In the end, the CodeAgent will have to return a final answer using the `final_answer` tool

Whenever you want to point out anything in the trace, cite the span_id number of the span that you are referring to.
"""